In [13]:
%load_ext autoreload
%autoreload 2

# BIOMD0000000955 - https://pmc.ncbi.nlm.nih.gov/articles/PMC7175834/
# BIOMD0000000956 - https://pmc.ncbi.nlm.nih.gov/articles/PMC7382213/
# BIOMD0000000957 - https://pmc.ncbi.nlm.nih.gov/articles/PMC7104073/
# BIOMD0000000958 - https://pmc.ncbi.nlm.nih.gov/articles/PMC7184012/
# BIOMD0000000960 - https://pmc.ncbi.nlm.nih.gov/articles/PMC7394373/

In [8]:
import pandas as pd

# Old TA1 Extraction results 
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ2G-ZFSotS2qo94mnTWCDvj9Y49ai-9O61DA7940sPYynEdBXq2cT2-Wl3nNldIb3gkpbPFaTFY2PJ/pub?output=xlsx"
xls = pd.ExcelFile(url)
xls

In [9]:
sheets = {
    sheet_name: pd.read_excel(xls, sheet_name)
    for sheet_name in xls.sheet_names
}
for sheet in sheets.values():
    sheet['grounding'] = sheet['grounding'].map(eval)
set(sheets)

{'BIOMD0000000955',
 'BIOMD0000000956',
 'BIOMD0000000957',
 'BIOMD0000000958',
 'BIOMD0000000960'}

In [15]:
# load new groundings stored in version control 
from collections import defaultdict
ground_truth_df = pd.read_csv("../mira/resources/mapped_biomodels_groundings.csv", sep=',')
dd = defaultdict(set)
for row_index,entry in ground_truth_df.iterrows():
    model_id = entry["model"]
    if not pd.isna(entry["identifiers"]):
        curie = entry["identifiers"].split("/")[0]
    else:
        curie = entry["mapped_identifiers"].split("/")[0]

    dd[model_id].add(curie)
dd = dict(dd)
dd

{'BIOMD0000000955': {'ido:0000511',
  'ido:0000514',
  'ido:0000592',
  'ncit:C28554'},
 'BIOMD0000000991': {'ido:0000511',
  'ido:0000514',
  'ido:0000592',
  'ncit:C28554'},
 'BIOMD0000000977': {'ido:0000511', 'ido:0000514', 'ido:0000592'},
 'BIOMD0000000971': {'ido:0000511', 'ido:0000514', 'ido:0000592'},
 'BIOMD0000000960': {'ido:0000511',
  'ido:0000514',
  'ido:0000621',
  'ncit:C28554'},
 'BIOMD0000000958': {'ido:0000511',
  'ido:0000514',
  'ido:0000597',
  'ncit:C25179',
  'ncit:C25746',
  'ncit:C28554'},
 'BIOMD0000000972': {'ido:0000511', 'ido:0000514', 'ido:0000592'},
 'BIOMD0000000981': {'ido:0000511',
  'ido:0000514',
  'ido:0000621',
  'ncit:C28554'},
 'BIOMD0000000957': {'ido:0000511',
  'ido:0000514',
  'ido:0000621',
  'ncit:C15220'},
 'BIOMD0000000962': {'ido:0000511', 'ido:0000514'},
 'BIOMD0000000988': {'efo:0001460',
  'ido:0000511',
  'ido:0000514',
  'ido:0000592',
  'ncit:C28554'},
 'BIOMD0000000974': {'ido:0000511',
  'ido:0000514',
  'ido:0000592',
  'ncit:C2

In [11]:
# Ta1 Extraction result table 
rows = []
for model_id, sheet in sheets.items():
    sheet_curies = {
        grounding["id"]
        for groundings in sheet["grounding"]
        for grounding in groundings
    }
    ground_truth_curies = dd[model_id]

    intersection_curies = sheet_curies & ground_truth_curies
    n_intersection = len(intersection_curies)
    n_ground_truth = len(ground_truth_curies)
    n_sheet = len(sheet_curies)
    rows.append((
        model_id,
        n_ground_truth,
        n_intersection,
        round(n_intersection / n_ground_truth, 2)
    ))

print("The way to interpret the precision is the percent")
pd.DataFrame(rows, columns=["model", "structured", "TA-1", "precision"])

The way to interpret the precision is the percent


,model,structured,TA-1,precision
0,BIOMD0000000955,4,0,0.00
1,BIOMD0000000956,3,0,0.00
2,BIOMD0000000957,4,1,0.25
3,BIOMD0000000958,6,1,0.17
4,BIOMD0000000960,4,0,0.00


In [14]:
from mira.openai import OpenAIClient
from mira.sources.sympy_ode.llm_util import execute_template_model_from_sympy_odes, image_file_to_odes_str
from IPython.display import Image

client = OpenAIClient()

In [16]:
ode_str0 = image_file_to_odes_str('images/BIOMD0000000955_odes.png', client=client)
tm0 = execute_template_model_from_sympy_odes(ode_str=ode_str0, attempt_grounding=True, client=client)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
ode_str1 = image_file_to_odes_str('images/BIOMD0000000956_odes.png', client=client)
tm1 = execute_template_model_from_sympy_odes(ode_str=ode_str1, attempt_grounding=True, client=client)

In [ ]:
ode_str2 = image_file_to_odes_str('images/BIOMD0000000957_odes.png', client=client)
tm2 = execute_template_model_from_sympy_odes(ode_str=ode_str2, attempt_grounding=True, client=client)

In [ ]:
ode_str3 = image_file_to_odes_str('images/BIOMD0000000958_odes.png', client=client)
tm3 = execute_template_model_from_sympy_odes(ode_str=ode_str3, attempt_grounding=True, client=client)

In [ ]:
ode_str4 = image_file_to_odes_str('images/BIOMD0000000960_odes.png', client=client)
tm4 = execute_template_model_from_sympy_odes(ode_str=ode_str4, attempt_grounding=True, client=client)